# SQL 

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

### Описание данных

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

### Логика базы данных


<img src=https://i.ibb.co/M14ThkM/jpg.jpg><br>



## Оглавление
1. [Цели исследования](#step1)  
  
        
2. [Подключение к базе данных](#step2)  
 
    
3. [Исследуем таблицы](#step3)  


4. [Сколько книг вышло после 1 января 2000 года](#step4)  


5. [Для каждой книги считаем количество обзоров и среднюю оценку](#step5)  


6. [Определяем издательство, которое выпустило наибольшее число книг толще 50 страниц](#step6)  


7. [Определяем автора с самой высокой средней оценкой книг](#step7)  


8. [Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок](#step8)  


9. [Общий вывод](#step9)




### Цели исследования <a id="step1"></a>   
</div>

- Собрать аналитику по книгам, вышедшим в 21 веке (количество книг, среднюю оценку, количество обзоров);
- Отсеять от аналитики мелкие книги и брошюры;
- После предыдущего этапа найти самое крупное издательство по выпущенным книгам;
- Найти активных пользователей и, возможно, лидеров мнений;
- **Дать общие рекомендации и задать направление развития для нашего сервиса.**

### Подключение к базе данных <a id="step2"></a>   
</div>

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
     'pwd': 'Sdf4$2;d-d30pp', # пароль
     'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
     'port': 6432, # порт подключения
     'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                     db_config['pwd'],
                     db_config['host'],
                     db_config['port'],
                     db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

In [2]:
pd.set_option('display.expand_frame_repr', True)

### Исследуем таблицы <a id="step3"></a>   
</div>

Напишем функцию для вывода таблиц

**Таблица books**

In [3]:
query = '''
    SELECT 
        *
    FROM 
        books
    
'''
pd.io.sql.read_sql(query, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
...,...,...,...,...,...,...
995,996,571,Wyrd Sisters (Discworld #6; Witches #2),265,2001-02-06,147
996,997,454,Xenocide (Ender's Saga #3),592,1996-07-15,297
997,998,201,Year of Wonders,358,2002-04-30,212
998,999,94,You Suck (A Love Story #2),328,2007-01-16,331


**Таблица authors**

In [4]:
query = '''
    SELECT 
        *
    FROM 
        authors
    
'''
pd.io.sql.read_sql(query, con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd
...,...,...
631,632,William Strunk Jr./E.B. White
632,633,Zadie Smith
633,634,Zilpha Keatley Snyder
634,635,Zora Neale Hurston


**Таблица ratings**

In [5]:
query = '''
    SELECT 
        *
    FROM 
        ratings
    
'''
pd.io.sql.read_sql(query, con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2
...,...,...,...,...
6451,6452,1000,carolrodriguez,4
6452,6453,1000,wendy18,4
6453,6454,1000,jarvispaul,5
6454,6455,1000,zross,2


**Таблица reviews**

In [6]:
query = '''
    SELECT 
        *
    FROM 
        reviews
    
'''
pd.io.sql.read_sql(query, con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...
...,...,...,...,...
2788,2789,999,martinadam,Later hospital turn easy community. Fact same ...
2789,2790,1000,wknight,Change lose answer close pressure. Spend so now.
2790,2791,1000,carolrodriguez,Authority go who television entire hair guy po...
2791,2792,1000,wendy18,Or western offer wonder ask. More hear phone f...


**Таблица publishers**

In [7]:
query = '''
    SELECT 
        *
    FROM 
        publishers
    
'''
pd.io.sql.read_sql(query, con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company
...,...,...
335,336,Workman Publishing Company
336,337,Wyatt Book
337,338,Yale University Press
338,339,Yearling


Таким образом у нас:
- 1000 книг;
- 636 авторов;
- 6456 оценок;
- 2793 обзоров;
- 340 издательств.

<img src=https://i.ytimg.com/vi/7-Kv2-CspbA/maxresdefault.jpg><br>


### Сколько книг вышло после 1 января 2000 года <a id="step4"></a>   
</div>

In [8]:
query = '''
    SELECT 
        COUNT(title)AS "Количество книг с 01.01.2000"
    FROM 
        books
    WHERE 
        CAST(publication_date AS date) > '1999-12-31'
        
    
'''
pd.io.sql.read_sql(query, con = engine)

,Количество книг с 01.01.2000
0,821


А кажется, что книг пишут больше... :) 

### Для каждой книги считаем количество обзоров и среднюю оценку <a id="step5"></a>   
</div>

In [9]:
query = '''
    SELECT
        SUBQ1.book_id as "book_id",
        SUBQ1.book_title as "Название книги",
        SUBQ2.cnt_reviews as "Количество обзоров",
        ROUND(SUBQ1.avg_rating,3) as "Средний рейтинг"
        
    FROM
        (SELECT
            books.book_id as book_id,
            books.title as book_title,
            AVG(ratings.rating) as avg_rating
        FROM
            books
        LEFT JOIN ratings ON books.book_id = ratings.book_id
        
        GROUP BY
            books.book_id,
            books.title
        ) AS SUBQ1
    LEFT JOIN (
        SELECT
            books.book_id as book_id,
            COUNT(reviews.review_id) as cnt_reviews
        FROM
            books
        LEFT JOIN reviews ON books.book_id = reviews.book_id
        
        GROUP BY
            books.book_id,
            books.title) AS SUBQ2 on SUBQ2.book_id = SUBQ1.book_id
    GROUP BY
        SUBQ1.book_id,
        SUBQ1.book_title,
        SUBQ2.cnt_reviews,
        SUBQ1.avg_rating
    
    ORDER BY
        "Количество обзоров" DESC
    LIMIT
        15
    
'''
pd.io.sql.read_sql(query, con = engine)

,book_id,Название книги,Количество обзоров,Средний рейтинг
0,948,Twilight (Twilight #1),7,3.663
1,779,The Lightning Thief (Percy Jackson and the Oly...,6,4.081
2,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.288
3,497,Outlander (Outlander #1),6,4.125
4,696,The Da Vinci Code (Robert Langdon #2),6,3.831
5,854,The Road,6,3.773
6,750,The Hobbit or There and Back Again,6,4.125
7,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.415
8,963,Water for Elephants,6,3.977
9,656,The Book Thief,6,4.264


In [10]:
query = '''
    SELECT
        SUM(SUBQ3.cnt_reviews) as sum_reviews
    FROM (
    SELECT
        SUBQ1.book_id as "book_id",
        SUBQ1.book_title as "Название книги",
        SUBQ2.cnt_reviews as cnt_reviews,
        ROUND(SUBQ1.avg_rating,3) as "Средний рейтинг"
        
    FROM
        (SELECT
            books.book_id as book_id,
            books.title as book_title,
            AVG(ratings.rating) as avg_rating
        FROM
            books
        LEFT JOIN ratings ON books.book_id = ratings.book_id
        
        GROUP BY
            books.book_id,
            books.title
        ) AS SUBQ1
    LEFT JOIN (
        SELECT
            books.book_id as book_id,
            COUNT(reviews.review_id) as cnt_reviews
        FROM
            books
        LEFT JOIN reviews ON books.book_id = reviews.book_id
        
        GROUP BY
            books.book_id,
            books.title) AS SUBQ2 on SUBQ2.book_id = SUBQ1.book_id
    GROUP BY
        SUBQ1.book_id,
        SUBQ1.book_title,
        SUBQ2.cnt_reviews,
        SUBQ1.avg_rating
    
    ORDER BY
        "cnt_reviews" DESC) AS SUBQ3
    
    
'''
pd.io.sql.read_sql(query, con = engine)

,sum_reviews
0,2793.0


Всё-таки **Сумерки**, целых **7 обзоров**

<img src=https://avatars.mds.yandex.net/get-turbo/1385617/2a00000167422deff89daf6e61e74b3c57d2/max_g480_c12_r4x3_pd10><br>

P.P.S: Да, знаю, что фильм и книги разное, но мы хейтеры - такие хейтеры :D


### Определяем издательство, которое выпустило наибольшее число книг толще 50 страниц <a id="step6"></a>   
</div>

In [11]:
query = '''
    SELECT 
        publishers.publisher as "Издательство",
        COUNT(books.book_id) as "Количество книг"
    
    FROM 
        publishers

    INNER JOIN books ON publishers.publisher_id = books.publisher_id   
    
    WHERE 
        books.num_pages > 50 
    
    GROUP BY
        publishers.publisher

    ORDER BY
        "Количество книг" DESC

    LIMIT
        20

'''
pd.io.sql.read_sql(query, con = engine)

,Издательство,Количество книг
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Bantam,19
5,Ballantine Books,19
6,Berkley,17
7,Berkley Books,14
8,St. Martin's Press,14
9,William Morrow Paperbacks,13


И наш победитель: **Британское издательство - *Penguin Books***.  

Плюс к этому, **Penguic Classic** - также входит их холдинг

<img src=https://upload.wikimedia.org/wikipedia/ru/3/33/Penguin_logo.png><br>
    
    

<div class="alert alert-block alert-warning">
<b>Комментарии от ревьюера:</b>

Согласен с результатом. Действительно данное издательство является лидером по выпущенным книгам. Но надо убрать фильтр по дате
</div>

<div class="alert alert-info">
<h2> Комментарий студента</h2>

Фильтр убрал, думал он относится ко всему исследованию, поэтому даже ниже сделал в двух вариантах, когда получил "скучный" ответ

</div>

### Определяем автора с самой высокой средней оценкой книг <a id="step7"></a>   
</div>

*Учитываем только книги с 50 и более оценками*

In [12]:
query = '''
    SELECT 
        SUBQ2.author as "Имя автора",
        AVG(SUBQ1.avg_rating) as "Средняя оценка",
        SUM(SUBQ1.cnt_rating) as "Количество оценок"
    FROM
        (SELECT
            books.book_id as book_id,
            books.author_id as author_id,
            AVG(ratings.rating) as avg_rating,
            COUNT(ratings.rating) as cnt_rating
        FROM
            books
        LEFT JOIN
            ratings ON books.book_id = ratings.book_id
             
        GROUP BY
            books.book_id
            
        HAVING
            COUNT(ratings.rating)>= 50) AS SUBQ1
    LEFT JOIN
        (SELECT
            authors.author as author,
            authors.author_id as author_id
        FROM
            authors
        LEFT JOIN
            books ON authors.author_id = books.author_id
        GROUP BY
            authors.author,
            authors.author_id
            ) AS SUBQ2 on SUBQ1.author_id = SUBQ2.author_id
    GROUP BY
        SUBQ2.author
        


    ORDER BY
        "Средняя оценка" DESC
'''
pd.io.sql.read_sql(query, con = engine)   

,Имя автора,Средняя оценка,Количество оценок
0,J.K. Rowling/Mary GrandPré,4.283844,310.0
1,Markus Zusak/Cao Xuân Việt Khương,4.264151,53.0
2,J.R.R. Tolkien,4.258446,162.0
3,Louisa May Alcott,4.192308,52.0
4,Rick Riordan,4.080645,62.0
5,William Golding,3.901408,71.0
6,J.D. Salinger,3.825581,86.0
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474,57.0
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879,66.0
9,Dan Brown,3.754540,143.0


Абсолютный победитель -  **Джоан Роулинг**.  
Совсем рядом с ней **Маркус Зусак**.   
**Толкин** - топ-3. 

In [13]:
query = '''
SELECT
            books.book_id as book_id,
            books.title as title,
            books.author_id as author_id,
            AVG(ratings.rating) as avg_rating,
            COUNT(ratings.rating) as cnt_rating
        FROM
            books
        LEFT JOIN
            ratings ON books.book_id = ratings.book_id
             
        GROUP BY
            books.book_id
        HAVING
            COUNT(ratings.rating)>= 50
            
      
'''
pd.io.sql.read_sql(query, con = engine)        

,book_id,title,author_id,avg_rating,cnt_rating
0,75,Angels & Demons (Robert Langdon #1),106,3.678571,84
1,750,The Hobbit or There and Back Again,240,4.125000,88
2,545,Romeo and Juliet,630,3.787879,66
3,948,Twilight (Twilight #1),554,3.662500,160
4,488,Of Mice and Men,311,3.622951,61
5,696,The Da Vinci Code (Robert Langdon #2),106,3.830508,59
6,722,The Fellowship of the Ring (The Lord of the Ri...,240,4.391892,74
7,627,The Alchemist,469,3.789474,57
8,733,The Giver (The Giver #1),372,3.750000,56
9,779,The Lightning Thief (Percy Jackson and the Oly...,498,4.080645,62


In [14]:
query = '''
       SELECT
            SUBQ1.book_id as book_id,
            authors.author as author,
            SUBQ1.title as title,
            SUBQ1.author_id as author_id,
            SUBQ1.avg_rating as avg_rating,
            SUBQ1.cnt_rating as cnt_rating
       FROM
           authors
       RIGHT JOIN
       (SELECT
            books.book_id as book_id,
            books.title as title,
            books.author_id as author_id,
            AVG(ratings.rating) as avg_rating,
            COUNT(ratings.rating) as cnt_rating
        FROM
            books
        LEFT JOIN
            ratings ON books.book_id = ratings.book_id
             
        GROUP BY
            books.book_id
        HAVING
            COUNT(ratings.rating)>= 50) SUBQ1 ON SUBQ1.author_id = authors.author_id
        GROUP BY
        SUBQ1.book_id,
        authors.author,
        SUBQ1.title,
        SUBQ1.author_id,
        SUBQ1.avg_rating,
        SUBQ1.cnt_rating 
            

'''
pd.io.sql.read_sql(query, con = engine)   

,book_id,author,title,author_id,avg_rating,cnt_rating
0,779,Rick Riordan,The Lightning Thief (Percy Jackson and the Oly...,498,4.080645,62
1,79,George Orwell/Boris Grabnar/Peter Škerl,Animal Farm,195,3.729730,74
2,488,John Steinbeck,Of Mice and Men,311,3.622951,61
3,299,J.K. Rowling/Mary GrandPré,Harry Potter and the Chamber of Secrets (Harry...,236,4.287500,80
4,696,Dan Brown,The Da Vinci Code (Robert Langdon #2),106,3.830508,59
5,627,Paulo Coelho/Alan R. Clarke/Özdemir İnce,The Alchemist,469,3.789474,57
6,733,Lois Lowry,The Giver (The Giver #1),372,3.750000,56
7,405,William Golding,Lord of the Flies,621,3.901408,71
8,673,J.D. Salinger,The Catcher in the Rye,235,3.825581,86
9,948,Stephenie Meyer,Twilight (Twilight #1),554,3.662500,160


In [15]:
# КОД ПОДЗАПРОСА

query = '''
        SELECT
            books.author_id as author_id,
            AVG(ratings.rating) as avg_rating,
            COUNT(ratings.rating) as cnt_rating
        FROM
            books
        LEFT JOIN
            ratings ON books.book_id = ratings.book_id
             
        GROUP BY
            author_id
        HAVING
            COUNT(ratings.rating)>= 50
'''
pd.io.sql.read_sql(query, con = engine)   

,author_id,avg_rating,cnt_rating
0,554,3.662500,160
1,257,3.859375,64
2,630,3.787879,66
3,302,3.971429,70
4,3,4.283019,53
5,39,4.107143,56
6,498,4.130952,84
7,469,3.789474,57
8,130,4.300000,50
9,236,4.288462,312


### Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок <a id="step8"></a>   
</div>

Для начала просто найдём самым активных пользователей. Активных нужно знать в никнейм

In [16]:
query = '''
    SELECT
        SUBQ1.username as "Активные пользователи",
        SUBQ1.cnt_reviews as "Количество обзоров",
        SUBQ2.cnt_rating as "Количество оценок"
    FROM
        (SELECT   
            reviews.username as username,
            COUNT(reviews.review_id) as cnt_reviews
        FROM
            reviews
        INNER JOIN books ON reviews.book_id = books.book_id
    
        WHERE 
            CAST(books.publication_date AS date) > '1999-12-31'  
        
        GROUP BY
            reviews.username
  
        ORDER BY
            cnt_reviews DESC) as SUBQ1
        
    INNER JOIN
        (SELECT
            ratings.username as username,
            COUNT(ratings.rating) as cnt_rating
            
         FROM
            ratings
         INNER JOIN books ON ratings.book_id = books.book_id
         
         WHERE 
            CAST(books.publication_date AS date) > '1999-12-31'      
         GROUP BY
            ratings.username
         HAVING
            COUNT(ratings.rating) > 50) as SUBQ2 ON SUBQ2.username = subq1.username   
    ORDER BY
        "Количество обзоров" DESC

    

'''
pd.io.sql.read_sql(query, con = engine)

,Активные пользователи,Количество обзоров,Количество оценок
0,sfitzgerald,26,53


Даже обидно как-то, под наши суровые условия подходит только пользователь sfitzgerald. Он и >50 оценок оставил, и 26 обзоров написал.

Посмотрим что будет, если убрать фильтр ограничения по дате публикации книги

In [17]:
query = '''
    SELECT
        SUBQ1.username as "Активные пользователи",
        SUBQ1.cnt_reviews as "Количество обзоров",
        SUBQ2.cnt_rating as "Количество оценок"
    FROM
        (SELECT   
            reviews.username as username,
            COUNT(reviews.review_id) as cnt_reviews
        FROM
            reviews
        INNER JOIN books ON reviews.book_id = books.book_id
    

        
        GROUP BY
            reviews.username
  
        ORDER BY
            cnt_reviews DESC) as SUBQ1
        
    INNER JOIN
        (SELECT
            ratings.username as username,
            COUNT(ratings.rating) as cnt_rating
            
         FROM
            ratings
         INNER JOIN books ON ratings.book_id = books.book_id
         
   
         GROUP BY
            ratings.username
         HAVING
            COUNT(ratings.rating) > 50) as SUBQ2 ON SUBQ2.username = subq1.username   
    ORDER BY
        "Количество обзоров" DESC

    

'''
pd.io.sql.read_sql(query, con = engine)

,Активные пользователи,Количество обзоров,Количество оценок
0,sfitzgerald,28,55
1,martinadam,27,56
2,richard89,26,55
3,jennifermiller,25,53
4,paul88,22,56
5,xdavis,18,51


Здесь уже пользователей побольше, поэтому посчитаем их среднее количество обзоров

In [18]:
query = '''
    SELECT
        ROUND(AVG(SUBQ3.cnt_review),2) as "Среднее количество обзоров"
    FROM(    
    SELECT
        SUBQ1.username as "Активные пользователи",
        SUBQ1.cnt_reviews as cnt_review,
        SUBQ2.cnt_rating as "Количество оценок"
    FROM
        (SELECT   
            reviews.username as username,
            COUNT(reviews.review_id) as cnt_reviews
        FROM
            reviews
        INNER JOIN books ON reviews.book_id = books.book_id
    

        
        GROUP BY
            reviews.username
  
        ORDER BY
            cnt_reviews DESC) as SUBQ1
        
    INNER JOIN
        (SELECT
            ratings.username as username,
            COUNT(ratings.rating) as cnt_rating
            
         FROM
            ratings
         INNER JOIN books ON ratings.book_id = books.book_id
         
   
         GROUP BY
            ratings.username
         HAVING
            COUNT(ratings.rating) > 50) as SUBQ2 ON SUBQ2.username = subq1.username   
    ORDER BY
        cnt_review DESC) as SUBQ3

    

'''
display(pd.io.sql.read_sql(query, con = engine))

,Среднее количество обзоров
0,24.33


### Общий вывод <a id="step9"></a>   
</div>

- С 01 января 2000 года выпущена **821 книга**;
- Самая обозреваемая книга - **Сумерки - 7 обзоров**;
- Самое активное издательство - **холдинг Penguins**;
- Самый высокая оценка у книг **Джоан Роулинг**;
- Пользователь, написавший больше всего отзывов с 01.01.2000 - **sfitzgerald**;
- В среднем наиболее активные пользователи пишут **24 отзыва**.


  

Для сервиса важно отметить то, что:
- Наш сервис не выживет без бестселлеров и больших саг;
- Важно не количество лайков или дизлайков для книги, а то, сколько обратной связи она собирает;
- Даже самые активные пользователи не пишут слишком много отзывов, всё-таки книги - не кино, поэтому важно правильно выстроить именно оценочную и рекомендательную автоматизированную систему, лидеры мнений вряд ли нам помогутю.